<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [580]:
import pandas as pd
import psycopg2

In [581]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [582]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [114]:
# текст запроса
query_3_1 = f'''
    Select 
        Count(*)
        
    From vacancies
'''

In [115]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [48]:
query_3_2 = f'''
    Select 
        Count(*)
    From employers
'''

In [49]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [50]:
query_3_3 = f'''
    Select 
        Count(*)
    From areas
'''

In [51]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [52]:
query_3_4 = f'''
    Select 
        Count(id)
    From industries
'''

In [53]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

Исходя из предварительного просмотра количества строк в таблицах (employers, vacansies, areas, industries), можно увидеть, что на все регионы (1362 шт.) у нас есть 23501 работодателей, которые в свою очередь предоставляю  49197 вакансий на 294 различных сфер деятельности. Из чего можно сделать вывод, на один регион в среднем 36 вакансий, что не так уж и много если учитывать только сайт hh.ru. На самом деле все обстоит иначе если мы посмотрим сколько вакансий в отдельно взятом регионе.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [66]:
query_4_1 = f'''
    Select 
        ar.name area, 
        Count(vac.id) cnt
    From areas As ar
    Join vacancies as vac On ar.id = vac.area_id
    Group By ar.id
    Order By 2 desc
'''

In [67]:
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\347208320.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [68]:
query_4_2 = f'''
    Select 
        Count(id)
    From vacancies
    Where (salary_from Is Not Null) Or (salary_to Is Not Null)
'''

In [69]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2168503709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [538]:
query_4_3 = f'''

    Select 
        Round(Avg(salary_from)) As min_salary, 
        Round(Avg(salary_to)) As max_salary
    From vacancies
    
'''

In [539]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2222372203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,min_salary,max_salary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [216]:
query_4_4 = f'''
    Select
        schedule, 
        employment,
        Count(*)
    From vacancies 
    Group By 1, 2
    Order By 3 desc
'''

In [217]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2981602351.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [218]:
query_4_5 = f'''
    Select 
        vac.experience,
        Count(*)
    From 
        vacancies As vac
        Join employers As emp On vac.employer_id = emp.id 
    Group By 1
    Order By 2
'''

In [219]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2601166264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Более детальный анализ вакансий предоставляет нам более понятную картину. Теперь мы можем убедиться, что большинство вакансий находиться в городах с большим населением, такие как: Москва, Санкт-Пербург, Минск, Новосибирск, Алматы. Так как у половины вакансий указана нижняя и вехняя вилка заработной платы, видно что у нижней границы она состовляет 71065 руб., а у верхней 110537 руб. соотвественно. Далее мы можем понять, что большая часть работодателей хочет, что бы их сотрудники работали по графику полный день, а трудоустройство занимало полную занятость. И на последок, опыт работы, в основном в вакансиях требуется опыт от 1 года до 3 лет, из этого можно сделать вывод, что на работу требуются люди с уже сформироваными и уверенными знаниями в своей сфере деятельности, но при этом эти люди все еще могут расти дальше в своей сфере.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [605]:
query_5_1 = f'''
    -- Вывод пятой строки
    (Select
        --Работодатель и количество вакансий
        emp.name,
        Count(*)
    From 
        --Данные из таблиц вакансий и работодателей соеденные через id
        vacancies As vac
        Join employers As emp On vac.employer_id = emp.id 
    Group By emp.id
    --Сортировка по убыванию
    Order By 2 Desc
    Offset 4 Limit 1)
    
    Union
    --Вывод первой строки
    (Select
        emp.name,
        Count(*)
    From 
        vacancies As vac
        Join employers As emp On vac.employer_id = emp.id 
    Group By emp.id
    --Сортировка по убыванию
    Order By 2 Desc
    Limit 1)
    --Общая сортировка двух строк
    Order By 2 Desc
'''

In [604]:
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\1792814825.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,name,count
0,Finstar Financial Group,9
1,ИДС Боржоми,9


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [586]:
query_5_2 = f'''
    Select
        -- Вывод региона и количесва работодателей и их вакансий
        ar.name As name,
        Count(distinct emp.id) As count_emp, 
        Count(vac.id) As count_vac 
    From 
        -- Данные из таблиц регионов, работодателей и вакансий 
        areas As ar
        Join employers As emp On ar.id = emp.area
        Left Join vacancies As vac On ar.id = vac.area_id
    -- Условие где нет вакансий
    Where vac.id is null
    Group By ar.id
    Having Count(distinct vac.id) = 0
    Order by 2 Desc, 3 Desc   

'''

In [587]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3341481329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,count_emp,count_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Витебская область,1,0
182,Ивацевичи,1,0
183,Дубровно,1,0
184,Буда-Кошелево,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [594]:
query_5_3 = f'''
    Select
        emp.name,
        Count(Distinct vac.area_id)
    From 
        vacancies As vac
        Join employers As emp On emp.id = vac.employer_id
    Group By  1, vac.employer_id
    Order By 2 Desc
'''

In [595]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\2398953733.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14901,Прогресс,1
14902,Прогресс,1
14903,Прогресс,1
14904,ПРОГРЕСС АГРО,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [592]:
query_5_4 = f'''
    Select
        Count(*)
    From 
        employers As emp
        Left Join employers_industries As emp_ind On emp_ind.employer_id = emp.id  
    -- Условие где у рабтодателей не указаны сферы деятельности
    Where emp_ind.industry_id Is Null
'''

In [593]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3114411492.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [317]:
query_5_5 = f'''
    Select
        --Уникальные имена компаний
        Distinct emp.name
    From 
        -- данные из employers и employers_industries присоедененной через id
        employers As emp
        Join employers_industries As emp_ind On emp.id = emp_ind.employer_id
    Group By emp.id
    -- Условие где количество сфер равно четырем
    Having Count(emp_ind.industry_id) = 4
    Order By 1
    Offset 2 Limit 1
'''

In [318]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\1650419958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [323]:
query_5_6 = f'''
    Select
        --Посчет количества работодателй
        Count(emp_ind.industry_id)
    From 
        --Данные из employrs и industries присоединенные через id
        employers_industries As emp_ind
        Join industries As ind On ind.id = emp_ind.industry_id
    --Условие выборки
    Where ind.name = 'Разработка программного обеспечения'
'''

In [324]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\1169039133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [358]:
#Не удавшаяся попытка парсинга данных из Wikipedia
""" from bs4 import BeautifulSoup
import requests

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

cities = []
table = soup.find('table', {'class': 'wikitable'})
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if cols:
        city = cols[1].text.strip()
        cities.append(city)

print(cities) """

Города-миллионеры России


In [560]:
query_5_7 = f'''
    (Select 
        --Регион и количество вакансий
        ar.name, 
        Count(*) As cnt
    From
        --Данные из регионов, вакансий и работодателей 
        areas As ar
        Join vacancies As vac On vac.area_id = ar.id
        Join employers As emp On emp.id = vac.employer_id
    --Условие выбора данных где есть компания в конкредных регионах
    Where (emp.name = 'Яндекс') and (ar.name in ('Москва','Санкт-Петербург','Новосибирск','Казань',
                                                'Волгоград','Екатеринбург','Нижний Новгород','Челябинск','Красноярск',
                                                'Самара','Уфа','Ростов-на-Дону','Омск','Краснодар','Воронеж','Пермь'))
    Group By ar.name
    Order By 2 
    )
    
    Union All
    -- Строка где все тоже самое, но выводиться общая сумма вакансий
    (Select
        'total', 
        Count(*)
    FROM 
        areas As ar
        Join vacancies As vac On vac.area_id = ar.id
        Join employers As emp On emp.id = vac.employer_id
    Where (emp.name = 'Яндекс') and (ar.name in ('Москва','Санкт-Петербург','Новосибирск','Казань',
                                                'Волгоград','Екатеринбург','Нижний Новгород','Челябинск','Красноярск',
                                                'Самара','Уфа','Ростов-на-Дону','Омск','Краснодар','Воронеж','Пермь'))
    )
    Order By cnt
'''

In [561]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3129374555.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Ростов-на-Дону,25
5,Пермь,25
6,Казань,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Благодаря анализу работодателей можно сразу понять, что по количеству вакансий у нас есть пять компаний: Яндекс, Ростелеком, Тинькофф, Сбер, Газпром нефть.
Конкретно меня интересовали первое(Яндекс) и пятое(Газпром нефть) место в этом списке. Далее я выяснил что у нас есть регионы в которых есть работодатели не разместившие вакансий на hh.ru. Так же в ходе выполнения заданий было найдено количество регионов в которых каждая компания разместила свои вакансии, среди которых у меня снова выделяется 'Яндекс' находящийся на первом месте этого списка. Далее я нашел количество компаний не указавших в своих вакансиях сферу деятельности (8419шт.), и на оборот компании у которых в указаных сферах деятельности была указана 'Разработка программного обеспечения' (3553шт.).
И на последок было произведено исследование количества вакансий у компании 'Яндекс', а конкретно сколько вакансий находиться в каждом регионе-миллионнике, где мы можем наблюдать тенденцию в зависимости от населения городов, меньше людей - меньше вакансий.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [390]:
query_6_1 = f'''
    Select
        vac.name
    From vacancies As vac
    Where vac.name ILike '%data%' Or vac.name ILike '%данн%'
'''

In [391]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\485020324.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей
...,...
1766,Программист Delphi / разработчик баз данных
1767,Разработчик баз данных (Big Data)
1768,Аналитик данных
1769,Data аналитик /Аналитик BI (удаленно)


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [566]:
query_6_2 = f'''
    Select
        Count(vac.id)
    From vacancies As vac
    --Условие выбора вакансий по ключивым словам
    Where (vac.name ILike '%data scientist%' Or vac.name ILike '%data science%' Or vac.name ILike '%исследователь данных%'
        Or (vac.name Like '%ML%' And vac.name Not Like '%HTML%') Or vac.name ILike '%machine learning%' Or vac.name ILike '%машинн%обучен%') 
        And (vac.name ILike '%junior%' Or vac.experience ILike '%нет опыта%' Or vac.employment ILike '%стажировка%')
'''

In [567]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3641650667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [474]:
query_6_3 = f'''
    Select
        Count(*)
    From vacancies As vac
    --Условие выбора вакансий по ключивым словам
    Where (vac.name ILike '%data scientist%' Or vac.name ILike '%data science%' Or vac.name ILike '%исследователь данных%'
        Or (vac.name Like '%ML%' And vac.name Not Like '%HTML%') Or vac.name ILike '%machine learning%' Or vac.name ILike '%машинн%обучен%') 
        And (vac.key_skills ILike '%sql%' Or vac.key_skills ILike '%postgres%')
'''

In [475]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\1159587760.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [476]:
query_6_4 = f'''
    Select
        Count(*)
    From vacancies As vac
    --Условие выбора вакансий по ключивым словам
    Where (vac.name ILike '%data scientist%' Or vac.name ILike '%data science%' Or vac.name ILike '%исследователь данных%'
        Or (vac.name Like '%ML%' And vac.name Not Like '%HTML%') Or vac.name ILike '%machine learning%' Or vac.name ILike '%машинн%обучен%') 
        And (vac.key_skills ILike '%python%')
'''

In [477]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\972386973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [484]:
query_6_5 = f'''
    Select
        --Формула посчета количества сфер деятельности
        Round(Avg(Length(vac.key_skills)-Length(Replace(vac.key_skills, CHR(9), ''))),2) + 1
    From vacancies As vac
    --Условие выбора вакансий по ключивым словам
    Where (vac.name ILike '%data scientist%' Or vac.name ILike '%data science%' Or vac.name ILike '%исследователь данных%'
        Or (vac.name Like '%ML%' And vac.name Not Like '%HTML%') Or vac.name ILike '%machine learning%' Or vac.name ILike '%машинн%обучен%')
'''

In [485]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\390248827.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,?column?
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [576]:
query_6_6 = f'''
        Select
            --Формала посчета средней ЗП 
            Round(Avg(coalesce((salary_from + salary_to)/2, salary_from, salary_to)))
        From vacancies As vac
        --Условие выбора вакансий по ключивым словам
        Where (vac.name ILike '%data scientist%' Or vac.name ILike '%data science%' Or vac.name ILike '%исследователь данных%'
            Or (vac.name Like '%ML%' And vac.name Not Like '%HTML%') Or vac.name ILike '%machine learning%' Or vac.name ILike '%машинн%обучен%')
            And ((vac.salary_from Is Not Null) Or (vac.salary_to Is Not Null))
            And (vac.experience ILike '%от 3 до 6 лет%')
''' 

In [577]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\user\AppData\Local\Temp\ipykernel_2072\3561045109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,round
0,243115.0


***

И в завершении предметный анализ в котором мы акцентируем свое внимание на вакансии Дата-Саентиста(далее DS). Тут можно сразу отметить что вакансий относящихся к данным у нас 1771шт., но только 51 вакансия у нас для начинающих(Junior) DS. Если нас интересует язык SQL и Postres, то в распаряжении есть 201 вакансия относящаяся к этому направлению. Так же DS специалисты используют язык Python, по этому было найдено 351 вакансия где этот язык был указан.
Далее, что бы лучше можно было понять сколько в среднем навыков нужно DS, при помощи фильтра выяснилось в среднем 6 навыков. Ну и самое вкусное на последок, это конечно же средняя зарплата опытного DS, ведь в будущем при усилении и расширении своих навыков, DS специалист с опытом работы от 3 до 6 лет может в среднем расчитывать на 243 115руб. 

# Общий вывод по проекту

Подводя итоги можно увидеть что рынок труда нуждается в большом количестве специалистов и компаниям требуются люди, что бы развиваться. Еще можно отметить, что так как большенство работодателей находяться в городах с большим населением, то вакансий в небольших городах гораздо меньше. Главные критерии при найме людей на работу это зарплата и график работы, и тут можно отметить хорошую оплату труда при полной занятости. Отдельного внимания заслуживает компания 'Яндекс' которая с хорошим отрывом преобладает на рынке труда по количеству вакансий и областей страны, в этом напралении можно исследовать среднее количество вакансий 'Яндекса' на область. Что касается будущих Data science специалистов, количество вакансий где нужно работать с данными, очень велико, что говорит о хорошем развитии это ветви программирования. Тут будет интересно продолжить исследование данных на наличие компаний готовых к найму людей с отсутсвием опыта работы и вычисление средней зарплаты для таких специалистов.